# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import openpyxl as Workbook

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2017, 9, 19)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData['Date'] = voltDailyData['Date'].dt.date

In [4]:
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
0,2017-06-25,163.7,52.6,3.03005
1,2017-06-26,202.4,42.5,4.15543
2,2017-06-27,202.3,53.8,3.97314
3,2017-06-28,197.7,52.4,3.93617
4,2017-06-29,192.6,45.2,3.83577


In [5]:
# Calculate total ICE (Internal Combustion Engine) miles driven
voltDailyData = voltDailyData.rename(columns = {'MilesDriven':'TotalMilesDriven'})
voltDailyData['TotalICEMiles'] = voltDailyData['TotalMilesDriven'] - voltDailyData['EvMilesDriven']
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles
0,2017-06-25,163.7,52.6,3.03005,111.1
1,2017-06-26,202.4,42.5,4.15543,159.9
2,2017-06-27,202.3,53.8,3.97314,148.5
3,2017-06-28,197.7,52.4,3.93617,145.3
4,2017-06-29,192.6,45.2,3.83577,147.4


In [6]:
# Create new DailyMPG Column, round values after adding new column
voltDailyData['DailyMPG'] = voltDailyData['TotalMilesDriven'] / voltDailyData['GallonsBurned']
voltDailyData['DailyMPG'] = pd.to_numeric(voltDailyData['DailyMPG'], errors='coerse')
voltDailyData = voltDailyData.replace(np.nan, 0)
voltDailyData = voltDailyData.round(2)
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.03
1,2017-06-26,202.4,42.5,4.16,159.9,48.71
2,2017-06-27,202.3,53.8,3.97,148.5,50.92
3,2017-06-28,197.7,52.4,3.94,145.3,50.23
4,2017-06-29,192.6,45.2,3.84,147.4,50.21


In [7]:
# Add '250+' miles to any DailyMPG value that has a value of infinity
# Why 250+? That's the "infinity mileage" that the volt displays in car
voltDailyData = voltDailyData.replace(np.inf, '250+')
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.03
1,2017-06-26,202.4,42.5,4.16,159.9,48.71
2,2017-06-27,202.3,53.8,3.97,148.5,50.92
3,2017-06-28,197.7,52.4,3.94,145.3,50.23
4,2017-06-29,192.6,45.2,3.84,147.4,50.21


In [8]:
voltDailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
Date                67 non-null object
TotalMilesDriven    67 non-null float64
EvMilesDriven       67 non-null float64
GallonsBurned       67 non-null float64
TotalICEMiles       67 non-null float64
DailyMPG            67 non-null object
dtypes: float64(4), object(2)
memory usage: 3.2+ KB


## Volt Readings

In [9]:
#Volt Readings throughout the day (usually two), convert Timestamp to datetime object
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltReadings.rename(columns = {'LocalTimestamp':'Date', 'GallonsBurned': 'LifeTimeGalBurned'}, inplace=True)
voltReadings['Date'] = pd.to_datetime(voltReadings['Date'])
voltReadings.head()

,BatteryLevel,ChargeState,OilLife,TirePressure_LF,TirePressure_LR,TirePressure_RF,TirePressure_RR,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-19 18:26:49,760.2289,64.89379,20635.22148,49334.13224,0.0,0.0,0.0,3.17535
1,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-18 19:30:13,760.2289,64.85196,20603.41349,49302.33046,0.0,0.0,0.0,3.17535
2,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-18 06:39:52,760.2289,64.80439,20567.26211,49266.17287,0.0,0.0,0.0,3.17535
3,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-17 18:44:08,760.2289,64.80438,20567.26211,49266.16044,0.0,0.0,0.0,3.21233
4,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-17 06:02:14,760.2289,64.80438,20567.26211,49266.16044,0.0,0.0,0.0,3.21233


In [10]:
# Select only columns desired
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned', 'LifetimeFuelEcon', 'LifetimeEvMiles','LifetimeMiles']]

In [11]:
# Calculate LifetimeICEMiles
voltReadings['LifetimeICEMiles'] = voltReadings['LifetimeMiles'] - voltReadings['LifetimeEvMiles']

In [12]:
# Sort based on timestamp, reorder columns, round values
voltReadings.sort_values(by='Date', inplace=True)
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned','LifetimeFuelEcon','LifetimeEvMiles','LifetimeICEMiles','LifetimeMiles']]
voltReadings = voltReadings.round(2)
voltReadings['Date'] = voltReadings.Date.dt.normalize()
voltReadings.head()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
167,2017-06-24,593.83,68.35,17941.91,22643.54,40585.45
166,2017-06-25,593.83,68.35,17941.91,22643.54,40585.45
165,2017-06-25,596.65,68.27,17988.50,22746.30,40734.80
164,2017-06-26,596.86,68.27,17994.46,22754.70,40749.16
163,2017-06-26,601.02,68.14,18036.93,22914.62,40951.55


In [13]:
# Drop dupes based on date column
voltReadings.drop_duplicates(subset='Date', keep='last', inplace=True)
voltReadings.head()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
167,2017-06-24,593.83,68.35,17941.91,22643.54,40585.45
165,2017-06-25,596.65,68.27,17988.50,22746.30,40734.80
163,2017-06-26,601.02,68.14,18036.93,22914.62,40951.55
161,2017-06-27,604.19,68.05,18084.05,23032.74,41116.79
159,2017-06-28,608.17,67.93,18130.94,23180.56,41311.50


In [14]:
# Combine dataframes based on date indexes
voltStats = pd.concat([voltReadings.set_index('Date'),voltDailyData.set_index('Date')], join='inner', axis=1)
voltStats.head()

,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
Date,,,,,,,,,,
2017-06-25,596.65,68.27,17988.50,22746.30,40734.80,163.7,52.6,3.03,111.1,54.03
2017-06-26,601.02,68.14,18036.93,22914.62,40951.55,202.4,42.5,4.16,159.9,48.71
2017-06-27,604.19,68.05,18084.05,23032.74,41116.79,202.3,53.8,3.97,148.5,50.92
2017-06-28,608.17,67.93,18130.94,23180.56,41311.50,197.7,52.4,3.94,145.3,50.23
2017-06-29,612.76,67.80,18188.35,23355.81,41544.16,192.6,45.2,3.84,147.4,50.21
2017-06-30,613.11,67.78,18188.35,23368.04,41556.39,12.2,0.0,0.35,12.2,35.25
2017-07-01,615.91,67.67,18221.60,23454.26,41675.86,119.5,33.3,2.80,86.2,42.64
2017-07-02,617.78,67.62,18247.15,23526.58,41773.73,97.9,25.5,1.87,72.4,52.34
2017-07-03,618.53,67.59,18256.11,23553.02,41809.13,35.4,9.0,0.75,26.4,47.35
